In [2]:
!pip install xgboost

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [4]:
df = pd.read_csv('atlantis_citizens_final.csv')
df['Commute_Out'] = (df['District_Name'] != df['Work_District']).astype(int)
df.shape

(15751, 11)

In [5]:
def remove_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df
df_clean= remove_outliers(df,['Wealth_Index','Life_Expectancy','House_Size_sq_ft'])
df_clean.shape

(10525, 11)

In [6]:
df_clean=df_clean.dropna()
print(df_clean.shape)
print(f"Cleaned data:\n{df_clean}\n")

(10525, 11)
Cleaned data:
      Citizen_ID       Diet_Type  District_Name Occupation  Wealth_Index  \
0      CIT_15935  Exotic Imports    Coral Slums     Scribe        1491.0   
1      CIT_11623         Seafood    Coral Slums     Fisher        1596.0   
2       CIT_8026         Seafood  Mariana Plaza    Warrior        3921.0   
5       CIT_7475         Seaweed  Mariana Plaza    Warrior        6173.0   
6      CIT_17545  Exotic Imports    Coral Slums     Scribe        1506.0   
...          ...             ...            ...        ...           ...   
15745  CIT_17305         Seaweed  Mariana Plaza     Scribe        6797.0   
15746  CIT_10602         Seaweed  Mariana Plaza   Merchant        2896.0   
15747   CIT_0865         Seaweed    Coral Slums      Miner        1671.0   
15748   CIT_0809  Exotic Imports  Mariana Plaza     Scribe        4656.0   
15750   CIT_6381         Seaweed    Coral Slums    Warrior        1439.0   

       House_Size_sq_ft  Life_Expectancy Vehicle_Owned  Work_

In [7]:
train = df_clean
test_original = pd.read_csv('test_atlantis_hidden.csv')
train= train.drop(['Citizen_ID','Bio_Hash','Work_District','Commute_Out'],axis=1)
test= test_original.drop(['Citizen_ID','Bio_Hash','Work_District'],axis=1)

In [8]:
train['Diet_Type'] = train['Diet_Type'].astype('category')
train['District_Name'] = train['District_Name'].astype('category')
train['Occupation'] = train['Occupation'].astype('category')
train['Vehicle_Owned'] = train['Vehicle_Owned'].astype('category')
x_train = train[['Diet_Type','District_Name','Wealth_Index','House_Size_sq_ft','Life_Expectancy','Vehicle_Owned']]
y_train = train['Occupation']

In [9]:
x_train.head()

,Diet_Type,District_Name,Wealth_Index,House_Size_sq_ft,Life_Expectancy,Vehicle_Owned
0,Exotic Imports,Coral Slums,1491.0,100.0,42.0,Fin Bicycle
1,Seafood,Coral Slums,1596.0,100.0,49.0,Sea Scooter
2,Seafood,Mariana Plaza,3921.0,533.0,37.0,Sea Scooter
5,Seaweed,Mariana Plaza,6173.0,911.0,47.0,Sea Scooter
6,Exotic Imports,Coral Slums,1506.0,100.0,51.0,No Vehicle


In [10]:
test['Diet_Type'] = test['Diet_Type'].astype('category')
test['District_Name'] = test['District_Name'].astype('category')
test['Vehicle_Owned'] = test['Vehicle_Owned'].astype('category')
x_test = test[['Diet_Type','District_Name','Wealth_Index','House_Size_sq_ft','Life_Expectancy','Vehicle_Owned']]

x_test['Diet_Type'] = le.fit_transform(x_test ['Diet_Type'])
x_test['District_Name'] = le.fit_transform(x_test['District_Name'])
x_test['Vehicle_Owned'] = le.fit_transform(x_test['Vehicle_Owned'])

In [11]:
model = XGBClassifier(
    objective='multi:softmax', 
    n_estimators=100, 
    learning_rate=0.1, 
    eval_metric='logloss',
    use_label_encoder=False,
    enable_categorical=True,
    tree_method="hist"
)

In [12]:
occupation_mapping = {
    "Warrior": 0,
    "Merchant": 1,
    "Fisher": 2,
    "Miner": 3,
    "Scribe": 4
}

y_train = train["Occupation"].map(occupation_mapping)

In [13]:
x_train.shape

(10525, 6)

In [14]:
model.fit(x_train, y_train)

C:\Users\medha\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:54:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,"objective objective: str | xgboost.sklearn._SklObjWProto | typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]] | NoneSpecify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'multi:softmax'
,"base_score base_score: float | typing.List[float] | NoneThe initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.List[xgboost.callback.TrainingCallback] | NoneList of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: float | NoneSubsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: float | NoneSubsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: float | NoneSubsample ratio of columns when constructing each tree.,None
,"device device: str | None.. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: int | None.. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,True
,"eval_metric eval_metric: str | typing.List[str | typing.Callable] | typing.Callable | None.. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes from sklearn.metrics import mean_absolute_error X, y = load_diabetes(return_X_y=True) reg = xgb.XGBRegressor( tree_method=

In [15]:
y_pred = model.predict(x_test)

In [16]:
y_prob = model.predict_proba(x_test)

In [17]:
print("\nPredicted probabilities (first 5 rows):\n", y_prob[:5])


Predicted probabilities (first 5 rows):
 [[0.00847912 0.078995   0.14209163 0.2046148  0.56581944]
 [0.00088929 0.08845041 0.01684603 0.0430119  0.8508023 ]
 [0.7772483  0.00124663 0.1733255  0.04683877 0.00134071]
 [0.21607952 0.00227917 0.6839876  0.09485348 0.00280029]
 [0.32631037 0.00246723 0.50641555 0.15095654 0.01385029]]


In [18]:
print(y_pred)

[4 4 0 ... 1 2 1]


In [19]:
submission = pd.DataFrame({"Citizen_ID":test_original['Citizen_ID'], "Occupation":y_pred})

In [20]:
submission.head()

,Citizen_ID,Occupation
0,CIT_15383,4
1,CIT_14830,4
2,CIT_17388,0
3,CIT_17438,2
4,CIT_16735,2


In [22]:
submission.to_csv("submission5.csv", index=False, quoting=csv.QUOTE_NONE)